# Imports

In [6]:
# %% Imports y configuración
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from pathlib import Path

# Agregar el directorio raíz al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

# Imports del proyecto
from model_ddp.utils.sistem_fun import (
    load_config,
    get_data_path,
    get_artifact_path,
    get_report_path,
    create_experiment_id,
    ensure_directories
)

from model_ddp.simulations.gaussian_simulator import (
    RegressionSimulator,
    SimulationConfig,
    RBFKernel,
    MaternKernel,
    PeriodicKernel,
    TransformationFunctions
)

from model_ddp.models.LSBP_normal_v1 import LSBPNormal

config=load_config()

# Flujo 

- Datos: Se guardaran todo en una carpeta. Por tanto todo tratamiento debe ir previo a un experimento.
- Experimentos: Cada modelo quedara en una carpeta con id, por tanto se crean versiones para cada uno.

In [7]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_lsbp_001"
SIM_REAL = "simulation"

In [8]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{NOMBRE_EJECUCION}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=500,
    n_features=5,
    x_range=(0.0, 10.0),
    noise_std=0.3,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=2.0,
    variance=1.0
)

# Definir transformación (regresión lineal)
coefficients = np.array([2.5, -1.8, 0.9, 1.2, -0.5])
intercept = 3.0

transformation = TransformationFunctions.linear(
    coefficients=coefficients,
    intercept=intercept
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (500, 5)
  Media por feature: [ 0.39226219 -0.71067439 -0.12112936 -0.29306343  1.00308722]
  Std por feature: [0.61994106 0.81604112 0.36195058 0.6161311  1.09806174]

Estadísticas de Y:
  Shape: (500,)
  Media: 4.2933
  Std: 2.3949
  Min: -2.3146
  Max: 8.1511
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\model_lsbp_001/_data.csv


## Experimento I 

In [ ]:
# Parámetros de ejecución
CARACTERISTICAS = "Test inicial: GP con kernel RBF + transformación lineal"
EXPERIMENT_ID = create_experiment_id("lsbp_001")

In [ ]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
lsbp_model = LSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)


# Ejecutar MCMC
trace = lsbp_model.run(
    iterations=50,          # Iteraciones totales
    burnin=10               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)

##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################



EJECUTANDO LSBPNormal...


IndexError: index 476 is out of bounds for axis 0 with size 476

In [ ]:
##################################################
# Graficas y FIT  (SEGUN EXPERIMENTO)
##################################################

In [ ]:
##################################################
# Reguistrar exprimento 
##################################################